In [1]:
import os
from botorch.models.gp_regression import FixedNoiseGP
from botorch.models.model_list_gp_regression import ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.utils.sampling import draw_sobol_samples
from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement,
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.optim.optimize import optimize_acqf
from botorch.acquisition.multi_objective.monte_carlo import qNoisyExpectedHypervolumeImprovement
from botorch.utils.multi_objective.box_decompositions.non_dominated import FastNondominatedPartitioning
from botorch.acquisition.monte_carlo import qNoisyExpectedImprovement
from botorch.acquisition.objective import GenericMCObjective
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.sampling import sample_simplex
from botorch.utils.transforms import normalize, unnormalize
from botorch.optim.optimize import optimize_acqf_list
import torch
import time
import warnings
from botorch import fit_gpytorch_mll
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.box_decompositions.dominated import DominatedPartitioning

########################################################################################################

tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}
SMOKE_TEST = os.environ.get("SMOKE_TEST")

# Define global variables (assuming tkwargs and NOISE_SE are already defined)
BATCH_SIZE = 4
NUM_RESTARTS = 10  # Adjust as needed
RAW_SAMPLES = 512  # Adjust as needed
ref_point = torch.tensor([-0.5, -0.2], **tkwargs)  # Reference point for qNEHVI
bounds = torch.stack([torch.zeros(1, **tkwargs), torch.ones(1, **tkwargs)])
standard_bounds = torch.zeros(2, 1, **tkwargs)
standard_bounds[1] = 1


########################################################################################################

def modified_sine_function_torch(x):
    return torch.sin(5 * torch.pi * x) * (1 - x) ** 2


def polynomial_exponential_function_torch(x):
    return x * torch.exp(-x ** 2) + 0.5 * torch.sin(4 * torch.pi * x)


########################################################################################################

NOISE_SE = torch.tensor([15.19, 0.63], **tkwargs)


def generate_initial_data(n=6):
    train_x = draw_sobol_samples(bounds=bounds, n=n, q=1).squeeze(1)
    train_obj1_true = modified_sine_function_torch(train_x)
    train_obj2_true = polynomial_exponential_function_torch(train_x)

    # Stack along the second dimension to match the shape of train_x
    train_obj_true = torch.stack([train_obj1_true, train_obj2_true], dim=-1).squeeze(-2)
    train_obj = train_obj_true + torch.randn_like(train_obj_true) * NOISE_SE
    return train_x, train_obj, train_obj_true


########################################################################################################

def initialize_model(train_x, train_obj, noise_se):
    models = []
    for i in range(train_obj.shape[-1]):  # Loop over each objective
        train_y = train_obj[..., i:i + 1]
        train_yvar = torch.full_like(train_y, noise_se[i] ** 2)

        # Initialize the GP model
        model = FixedNoiseGP(train_x, train_y, train_yvar)
        models.append(model)

    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return mll, model


########################################################################################################
BATCH_SIZE = 4
NUM_RESTARTS = 10 if not SMOKE_TEST else 2
RAW_SAMPLES = 512 if not SMOKE_TEST else 4

bounds = torch.stack([torch.zeros(1, **tkwargs), torch.ones(1, **tkwargs)])


def optimize_qehvi_and_get_observation(model, train_x, sampler, bounds):
    with torch.no_grad():
        pred = model.posterior(normalize(train_x, bounds)).mean
    partitioning = FastNondominatedPartitioning(
        ref_point=ref_point,
        Y=pred,
    )

    bounds = torch.stack([torch.zeros(1, **tkwargs), torch.ones(1, **tkwargs)])

    acq_func = qExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point,
        partitioning=partitioning,
        sampler=sampler,
    )

    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,
        options={"batch_limit": 5, "maxiter": 200},
        sequential=True,
    )

    new_x = unnormalize(candidates.detach(), bounds=bounds)
    new_obj_true = torch.stack([
        modified_sine_function_torch(new_x),
        polynomial_exponential_function_torch(new_x)
    ], -1)
    new_obj = new_obj_true + torch.randn_like(new_obj_true) * NOISE_SE
    return new_x, new_obj, new_obj_true


########################################################################################################

def optimize_qnehvi_and_get_observation(model, train_x, sampler):
    with torch.no_grad():
        pred = model.posterior(normalize(train_x, bounds)).mean
    partitioning = FastNondominatedPartitioning(
        ref_point=ref_point,
        Y=pred,
    )
    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point.tolist(),
        X_baseline=normalize(train_x, bounds),
        prune_baseline=True,
        sampler=sampler,
    )
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,
        options={"batch_limit": 5, "maxiter": 200},
        sequential=True,
    )
    new_x = unnormalize(candidates.detach(), bounds=bounds)
    new_obj_true = torch.stack([
        modified_sine_function_torch(new_x),
        polynomial_exponential_function_torch(new_x)
    ], -1)
    new_obj = new_obj_true + torch.randn_like(new_obj_true) * NOISE_SE
    return new_x, new_obj, new_obj_true


########################################################################################################

def optimize_qnparego_and_get_observation(model, train_x, sampler):

    train_x_normalized = normalize(train_x, bounds)
    with torch.no_grad():
        pred = model.posterior(train_x_normalized).mean
    acq_func_list = []
    for _ in range(BATCH_SIZE):
        weights = sample_simplex(2, **tkwargs).squeeze()  # 2 for two objectives
        objective = GenericMCObjective(
            lambda Y: get_chebyshev_scalarization(weights=weights, Y=Y)
        )
        acq_func = qNoisyExpectedImprovement(
            model=model,
            objective=objective,
            X_baseline=train_x_normalized,
            sampler=sampler,
            prune_baseline=True,
        )
        acq_func_list.append(acq_func)

    candidates, _ = optimize_acqf_list(
        acq_function_list=acq_func_list,
        bounds=standard_bounds,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,
        options={"batch_limit": 5, "maxiter": 200},
    )

    new_x = unnormalize(candidates.detach(), bounds=bounds)
    new_obj_true = torch.stack([
        modified_sine_function_torch(new_x),
        polynomial_exponential_function_torch(new_x)
    ], -1)
    new_obj = new_obj_true + torch.randn_like(new_obj_true) * NOISE_SE
    return new_x, new_obj, new_obj_true


########################################################################################################

warnings.filterwarnings("ignore", category=BadInitialCandidatesWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

N_BATCH = 3  # Adjust as needed
MC_SAMPLES = 128  # Adjust as needed
verbose = True

# Initialize lists to store hypervolume values
hvs_qparego, hvs_qehvi, hvs_qnehvi, hvs_random = [], [], [], []

# Generate initial training data and initialize model
train_x, train_obj, train_obj_true = generate_initial_data(n=2 * 1 + 2)  # Adjust as needed
print("train_x shape:", train_x.shape)
print("train_obj shape:", train_obj.shape)
mll, model = initialize_model(train_x, train_obj, NOISE_SE)

# Compute initial hypervolume
bd = DominatedPartitioning(ref_point=ref_point, Y=train_obj_true)
initial_volume = bd.compute_hypervolume().item()
hvs_qparego.append(initial_volume)
hvs_qehvi.append(initial_volume)
hvs_qnehvi.append(initial_volume)
hvs_random.append(initial_volume)

# Define samplers for acquisition functions
qparego_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))
qehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))
qnehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))

# Bayesian Optimization Loop
for iteration in range(1, N_BATCH + 1):
    t0 = time.monotonic()

    # Fit the models
    fit_gpytorch_mll(mll)

    # Optimize acquisition functions and get new observations
    new_x, new_obj_temp, new_obj_true_temp = optimize_qehvi_and_get_observation(
        model, train_x, qehvi_sampler, bounds  # Use the appropriate function and sampler
    )

    new_obj = new_obj_temp.view(-1, 2)

    new_obj_true = new_obj_true_temp.view(-1, 2)

    # Update training points
    train_x = torch.cat([train_x, new_x])
    train_obj = torch.cat([train_obj, new_obj])
    train_obj_true = torch.cat([train_obj_true, new_obj_true])

    # Reinitialize the model
    mll, model = initialize_model(train_x, train_obj, NOISE_SE)

    # Update hypervolume
    bd = DominatedPartitioning(ref_point=ref_point, Y=train_obj_true)
    volume = bd.compute_hypervolume().item()
    hvs_qnehvi.append(volume)  # Update the correct hypervolume list

    t1 = time.monotonic()
    if verbose:
        print(f"\nBatch {iteration:>2}: Hypervolume qNEHVI = {volume:>4.2f}, time = {t1 - t0:>4.2f}.")


/home/zengs/data/anaconda3/envs/venv_pl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


train_x shape: torch.Size([4, 1])
train_obj shape: torch.Size([4, 2])


/home/zengs/data/anaconda3/envs/venv_pl/lib/python3.8/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)
/home/zengs/data/anaconda3/envs/venv_pl/lib/python3.8/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)



Batch  1: Hypervolume qNEHVI = 0.74, time = 9.47.


/home/zengs/data/anaconda3/envs/venv_pl/lib/python3.8/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)



Batch  2: Hypervolume qNEHVI = 0.75, time = 4.88.

Batch  3: Hypervolume qNEHVI = 0.96, time = 5.42.


/home/zengs/data/anaconda3/envs/venv_pl/lib/python3.8/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


In [5]:
hvs_qnehvi

[0.6539858692428603,
 0.7382729687934243,
 0.7486096920491598,
 0.9590217860130419]